In [1]:
from flask import Flask, render_template
import pymysql
import pandas as pd
import folium
from folium.plugins import MarkerCluster
import matplotlib.pyplot as plt
import io
import base64

app = Flask(__name__)

# MySQL connection
def get_connection():
    return pymysql.connect(
        host="mysql.clarksonmsda.org",
        user="singarv",
        password="Clarkson23@#",
        database="singarv_accident",
        autocommit=True,
        cursorclass=pymysql.cursors.DictCursor
    )

In [2]:
@app.route('/map')
def accident_heatmap_with_tooltips():
    connection = get_connection()
    data = pd.read_sql("""
        SELECT Latitude, Longitude, Date, Weather_Conditions, Light_Conditions, Accident_Severity
        FROM Accidents
        WHERE Latitude != 0 AND Longitude != 0
    """, con=connection)
    connection.close()

    # Create a Folium map centered over the UK
    accident_map = folium.Map(location=[54.5260, -2.6189], zoom_start=6)

    # Add HeatMap layer (without tooltips)
    heat_data = [
        [row['Latitude'], row['Longitude'], 4 - row['Accident_Severity']]
        for _, row in data.iterrows()
    ]  # Weight: Fatal=3, Serious=2, Slight=1
    HeatMap(heat_data, radius=10, blur=15, max_zoom=10).add_to(accident_map)

    # Add tooltips for individual points
    for _, row in data.iterrows():
        tooltip = (
            f"<b>Date:</b> {row['Date']}<br>"
            f"<b>Weather:</b> {row['Weather_Conditions']}<br>"
            f"<b>Light Conditions:</b> {row['Light_Conditions']}<br>"
            f"<b>Severity:</b> {'Fatal' if row['Accident_Severity'] == 1 else 'Serious' if row['Accident_Severity'] == 2 else 'Slight'}"
        )
        folium.Marker(
            location=[row['Latitude'], row['Longitude']],
            popup=tooltip,
            icon=folium.Icon(color='red' if row['Accident_Severity'] == 1 else 'orange' if row['Accident_Severity'] == 2 else 'green')
        ).add_to(accident_map)

    # Save the map to the template folder
    accident_map.save('templates/map.html')
    return render_template('map.html')

In [3]:
@app.route('/')
def index():
    return render_template('index.html')

@app.route('/charts')
def charts():
    connection = get_connection()
    data = pd.read_sql("""
        SELECT Accident_Severity, COUNT(*) AS Count
        FROM Accidents
        GROUP BY Accident_Severity
    """, con=connection)
    connection.close()

    # Create bar chart
    import matplotlib.pyplot as plt
    import io
    import base64

    plt.figure(figsize=(8, 5))
    severity_labels = ['Fatal', 'Serious', 'Slight']
    plt.bar(severity_labels, data['Count'], color=['red', 'orange', 'green'])
    plt.xlabel('Accident Severity')
    plt.ylabel('Number of Accidents')
    plt.title('Accident Severity Distribution')

    # Save the plot to a PNG image
    img = io.BytesIO()
    plt.savefig(img, format='png')
    img.seek(0)
    plot_url = base64.b64encode(img.getvalue()).decode()

    return render_template('charts.html', plot_url=plot_url)

In [5]:
if __name__ == '__main__':
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1